# Training the tensorflow model on Google Cloud AI Platform
Use this notebook to either run models locally or in Google Cloud with or without using  AI Platform

In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [ ]:
!sudo chown -R jupyter:jupyter /home/jupyter/Breast-Cancer-Detection

In [ ]:
# without this, cv2 throws an error, install it before proceeding
# !sudo apt-get install ffmpeg libsm6 libxext6  -y

In [ ]:
import os

In [ ]:
# Google cloud details, not required if running on a local system
PROJECT = 'vernal-buffer-285906'
REGION = 'us-west1'
BUCKET = 'vernal-buffer-285906'

os.environ['BUCKET'] = BUCKET
os.environ['PROJECT'] = PROJECT
os.environ['REGION'] = REGION
os.environ['TFVERSION'] = '2.3'

In [ ]:
%%bash
gcloud config set project $PROJECT
gcloud config set compute/region $REGION

In [ ]:
# set train and output directories

# os.environ['TRAIN_DIR'] = "/home/jupyter/Breast-Cancer-Detection/data"
# os.environ['OUTPUT_DIR'] = "/home/jupyter/Breast-Cancer-Detection/model_results"

os.environ['TRAIN_DIR'] = 'gs://vernal-buffer-285906/test_cnn'
os.environ['OUTPUT_DIR'] = 'gs://vernal-buffer-285906/model_results'

## Running Locally

In [ ]:
%%bash

# export python path and run command. Bucket argument is optional, not required if not using Google Cloud machine
export PYTHONPATH=${PYTHONPATH}:/home/jupyter/Breast-Cancer-Detection/

# Train using python
python -m detectors.tf_gcp.trainer.task \
 --input-dir=$TRAIN_DIR \
 --output-dir=$OUTPUT_DIR \
 --bucket=$BUCKET \
 --num-epochs=50 \
 --batch_size=32 \
 --steps-per-epoch=113

## Submitting to Google Cloud AI Platform

In [ ]:
%%bash

# export python path and run command. Command submits job to AI platform. This can't be run on local machine. Has to be using Google Cloud AI notebooks.
export PYTHONPATH=${PYTHONPATH}:${PWD}/tf_trainer
    
OUTDIR=$OUTPUT_DIR
JOBNAME=breastcancer_$(date -u +%y%m%d_%H%M%S)
echo $OUTDIR $REGION $JOBNAME
gcloud ai-platform jobs submit training $JOBNAME \
  --region=$REGION \
  --module-name=trainer.task \
  --package-path=$(pwd)/tf_trainer/trainer \
  --job-dir=$OUTDIR \
  --staging-bucket=gs://$BUCKET \
  --scale-tier=BASIC \
  --runtime-version=2.3 \
  --python-version=3.7 \
  -- \
  --bucket=$BUCKET \
  --output-dir=$OUTDIR \
  --input-dir=$TRAIN_DIR \
  --num-epochs=50 \
  --batch_size=32 \
  --steps-per-epoch=113

## Monitoring Training using Tensorboard 

In [ ]:
# Instructions to run TensorBoard in google cloud

# Open the port you are supposed to use (usually 6006) in Google cloud vm instance
# info -> https://docs.bitnami.com/google/faq/administration/use-firewall/
# Run tensorboard using following command
# tensorboard --logdir <path/to/log/directory> --bind_all
# Ex:
# tensorboard --logdir gs://vernal-buffer-285906/model_results/tensorboard --bind_all